In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms

In [ ]:
transforms_train1 = torchvision.transforms.Compose([torchvision.transforms.RandomHorizontalFlip(p=0.5), 
                                                    torchvision.transforms.RandomVerticalFlip(p=0.5), 
                                                    torchvision.transforms.RandomGrayscale(p=0.5), 
                                                    torchvision.transforms.ToTensor()])
transforms_train2 = torchvision.transforms.Compose([torchvision.transforms.RandomEqualize(p=0.75), 
                                                    torchvision.transforms.RandomPerspective(distortion_scale=0.5, p=0.5), 
                                                    transforms.RandomCrop(32, padding=4), torchvision.transforms.ToTensor()])

trainingdata1 = torchvision.datasets.CIFAR10('./CIFAR10/', train=True, download=True, transform=transforms_train1)
trainingdata2 = torchvision.datasets.CIFAR10('./CIFAR10/', train=True, download=True, transform=transforms_train2)
trainingdata = trainingdata1 + trainingdata2
testingdata = torchvision.datasets.CIFAR10('./CIFAR10/', train=False, download=True, transform=torchvision.transforms.ToTensor())

In [ ]:
print(len(trainingdata))
trainDataLoader = torch.utils.data.DataLoader(trainingdata,batch_size=16,shuffle=True)
testDataLoader = torch.utils.data.DataLoader(testingdata,batch_size=16,shuffle=False)

In [ ]:
class BasicBlock(nn.Module):

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False), 
                nn.BatchNorm2d(planes))

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 128

        self.conv1 = nn.Conv2d(3, 128, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(128)
        self.layer1 = self._make_layer(block, 128, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 256, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 512, num_blocks[2], stride=2)
        self.dropout = nn.Dropout(p=0.5)
        self.linear = nn.Linear(2048, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def project1_model():
    return ResNet(BasicBlock, [1, 1, 1])

In [ ]:
net = project1_model().cuda()
Loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

net.load_state_dict(torch.load('./final_model.pt'))
net.train()

from torchsummary import summary
summary(net, (3,32,32))

In [ ]:
train_loss_history = []
test_loss_history = []
accuracy_history = []

for epoch in range(10):
    train_loss = 0.0
    test_loss = 0.0
    correct_outputs = 0.0

    for i, data in enumerate(trainDataLoader):
        images, labels = data
        images = images.cuda()
        labels = labels.cuda()
        optimizer.zero_grad()
        predicted_output = net(images)
        fit = Loss(predicted_output,labels)
        fit.backward()
        optimizer.step()
        train_loss += fit.item()

    for i, data in enumerate(testDataLoader):
        with torch.no_grad():
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            predicted_output = net(images)
            fit = Loss(predicted_output,labels)
            test_loss += fit.item()
            _, classifications = torch.max(predicted_output, dim=1)
            correct_outputs += torch.sum(classifications == labels).item()

    train_loss = train_loss/len(trainDataLoader)
    test_loss = test_loss/len(testDataLoader)
    train_loss_history.append(train_loss)
    test_loss_history.append(test_loss)
    if ((epoch == 0) or (correct_outputs/len(testingdata) > max(accuracy_history))):
        torch.save(net.state_dict(), './saved_models/3')
        print('saved!')
    accuracy_history.append(correct_outputs/len(testingdata))
    print("epochs done =", epoch, ', accuracy=', correct_outputs/len(testingdata))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot(range(10),train_loss_history,'-',linewidth=3,label='Train error')
ax1.plot(range(10),test_loss_history,'-',linewidth=3,label='Test error')
ax1.set_xlabel('epoch')
ax1.set_ylabel('loss')
ax1.grid(True)
ax1.legend(loc='upper left')

ax2.plot(range(10),accuracy_history,'g--',linewidth=1,label='Test accuracy')
ax2.set_ylabel('Accuracy')
ax2.set_ylim([0,1])
ax2.legend()

print('Max accuracy is:', max(accuracy_history))
# print(train_loss_history)
# print(test_loss_history)
# print(accuracy_history)